In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import v2
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
import utils
plt.ion()   # interactive mode

In [2]:
import presets, transforms, utils, sampler, train_quantization

In [3]:
import train as engine

In [4]:
from torchvision.models import MobileNet_V3_Large_Weights

In [5]:
torch.cuda.is_available()

True

In [6]:
device=torch.device('cuda')
print(f"Using {device} device")

Using cuda device


In [7]:
cudnn.enabled = True

In [8]:
cudnn.benchmark = True

In [9]:
transforms = {
    'train': v2.Compose([
        v2.AutoAugment(),
        v2.Resize(232),
        v2.CenterCrop(224),
        v2.PILToTensor(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        v2.RandomErasing(p=0.2)
    ]),
    'val': v2.Compose([
        #v2.AutoAugment(),
        v2.Resize(232),
        v2.CenterCrop(224),
        v2.PILToTensor(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        #v2.RandomErasing(p=0.2)
    ]),
}

In [10]:
transforms

{'train': Compose(
       AutoAugment(interpolation=InterpolationMode.NEAREST, policy=AutoAugmentPolicy.IMAGENET)
       Resize(size=[232], interpolation=InterpolationMode.BILINEAR, antialias=True)
       CenterCrop(size=(224, 224))
       PILToTensor()
       ToDtype(scale=True)
       Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=False)
       RandomErasing(p=0.2, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=[0.0], inplace=False)
 ),
 'val': Compose(
       Resize(size=[232], interpolation=InterpolationMode.BILINEAR, antialias=True)
       CenterCrop(size=(224, 224))
       PILToTensor()
       ToDtype(scale=True)
       Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=False)
 )}

In [11]:
data_="data"

In [12]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_, x),transforms[x]) for x in ['train', 'val']}

In [13]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=128,shuffle=True, num_workers=4)for x in ['train', 'val']}

In [14]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [15]:
class_names = image_datasets['train'].classes

In [16]:
class_names

['F-15 Eagle',
 'F-22 Raptor',
 'a-10 thunderbolt ii',
 'ac-130 ghostrider',
 'b-1 lancer',
 'b-2 spirit',
 'b-52 stratofortress',
 'f-35 lightning ii']

In [17]:
model=models.mobilenet_v3_large(weights=None)

In [18]:
model

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [19]:
num_ftrs = model.classifier[3].in_features
model.classifier[3] = nn.Linear(num_ftrs, len(class_names))

In [20]:
model

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [21]:
model = model.to(device)

In [22]:
criterion = nn.CrossEntropyLoss().cuda()
scaler=torch.amp.GradScaler('cuda',enabled=True)
# Observe that all parameters are being optimized
optimizer = optim.RMSprop(model.parameters(), lr=0.064, momentum=0.9, weight_decay=0.00001, eps=0.0316, alpha=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
lr_scheduler = lr_scheduler.StepLR(optimizer,gamma=0.973,step_size=2)

In [23]:
ema_decay=torch.optim.swa_utils.get_ema_multi_avg_fn(decay=0.99998)

In [24]:
ema=torch.optim.swa_utils.AveragedModel(model,device=device,multi_avg_fn=ema_decay)

In [25]:
class Methods:
    def __init__(self, print_freq, clip_grad_norm, lr_warmup_epochs, model_ema_steps):
        # Instance attributes
        self.print_freq = print_freq
        self.clip_grad_norm = clip_grad_norm
        self.lr_warmup_epochs = lr_warmup_epochs
        self.model_ema_steps = model_ema_steps

In [26]:
Args= Methods(1,1.0,5,32)

In [ ]:
for epoch in range(600):
    # train for one epoch, printing every 10 iterations
    engine.train_one_epoch(model, criterion, optimizer, dataloaders['train'], device,epoch,
                           Args, model_ema=ema, scaler=scaler)
    # update the learning rate
    #lr_scheduler.step()
    # evaluate on the test dataset
    engine.evaluate(model, criterion, dataloaders['val'], device, print_freq=1, log_suffix="")
    torch.save(model,'models/mobilenet.pt')
    torch.save(model.state_dict(),'models/mobilenet.pth')

C:\Users\arinz\image analysis\img_cls\mobilenet\train.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch: [0]  [0/5]  eta: 0:16:59  lr: 0.064  img/s: 0.8953239129392488  loss: 2.0845 (2.0845)  acc1: 10.9375 (10.9375)  acc5: 67.1875 (67.1875)  time: 203.9936  data: 61.0285  max mem: 2880
Epoch: [0]  [1/5]  eta: 0:07:27  lr: 0.064  img/s: 6.417542536154424  loss: 2.0253 (2.0549)  acc1: 10.9375 (13.2812)  acc5: 66.4062 (66.7969)  time: 111.9798  data: 30.5149  max mem: 2913
Epoch: [0]  [2/5]  eta: 0:04:03  lr: 0.064  img/s: 6.394804940615975  loss: 2.0845 (2.3653)  acc1: 10.9375 (11.1979)  acc5: 66.4062 (65.8854)  time: 81.3297  data: 20.3433  max mem: 2913
Epoch: [0]  [3/5]  eta: 0:02:11  lr: 0.064  img/s: 6.6351843377930555  loss: 2.0845 (2.3420)  acc1: 7.0312 (9.7656)  acc5: 64.8438 (65.6250)  time: 65.8226  data: 15.2576  max mem: 2913
Epoch: [0]  [4/5]  eta: 0:01:12  lr: 0.064  img/s: 1.053764085696587  loss: 2.0947 (2.2926)  acc1: 10.9375 (11.8123)  acc5: 64.8438 (65.2104)  time: 72.7785  data: 12.2061  max mem: 2913
Epoch: [0] Total time: 0:06:05
Test:   [0/5]  eta: 0:05:17  los